# 중급 프로젝트2
- **공유오피스 출입데이터**를 바탕으로 각 종 **비즈니스 지표의 추이를 예측**하여 **서비스의 개선 방향성을 제시**하는 프로젝트 입니다.
    - 공유오피스 서비스의 3일체험 신청, 신청자 일자별 방문 및 출입기록, 결제 여부, 지점별 면적 정보 등의 데이터가 담긴 데이터 셋이 제공됩니다.
    - 지금까지 학습했던 데이터 분석 방법론 및 머신러닝 기법을 활용하여 서비스의 현재 상태를 분석하고, 향후 사업적인 관점에서의 개선점을 제안해보세요.
        - ex1. 무료 유저의 유료 결제 전환율 예측 결과를 바탕으로 한 비즈니스 액션 제안
        - ex2. 유저의 오피스 이용량 시간 예측 결과로부터 도출한 개선안 제안
        - ex3. 유저의 방문일자 및 출입시간에 따른 방문 패턴 예측, 신규 상품 제안
- 프로젝트의 큰 주제 내에서 팀 별로 좀 더 관심이 있는 주제를 선정하고, 선정한 주제를 연구하는데 있어 필요한 정보가 무엇인지 파악하여 결론을 도출해보세요.

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 1. 데이터 확인

| #  | 테이블명            | 테이블 설명                         | 주요컬럼                                              | 비고                            |
|----|------------------|--------------------------------|--------------------------------------------------|-------------------------------|
| 1  | trial_register  | 3일체험 신청                      | 유저id, 3일체험신청일시                               | 3일 체험은 유저 당 1회만 제공         |
| 2  | trial_visit_info | 3일체험 신청자 일자별 방문기록         | 유저id, 날짜, 지점id, 최초입실시각, 최종퇴실시각, 체류시간 |                                  |
| 3  | trial_access_log | 3일체험 신청자 출입기록             | 유저id, 지점id, timestamp, 출입방향                 |                                  |
| 4  | trial_payment   | 3일체험 신청자 결제 여부             | 유저id, 결제여부                                    |                                  |
| 5  | site_area       | 지점별 면적                        | 지점id, 지점 면적                                    |                                  |


In [ ]:
site_area_df = pd.read_csv("../01_원본데이터/site_area.csv")
trial_access_log = pd.read_csv("../01_원본데이터/trial_access_log.csv")
trial_payment = pd.read_csv("../01_원본데이터/trial_payment.csv")
trial_register = pd.read_csv("../01_원본데이터/trial_register.csv")
trial_visit_info = pd.read_csv("../01_원본데이터/trial_visit_info.csv")

### 1. 지점별 면적 데이터 분석
- 총 지점의 개수 : 9개
- 지점 평수 : 50평, 100평, 150평

In [ ]:
# 지점별 면적
site_area_df

,site_id,area_pyeong
0,1,50
1,2,100
2,3,150
3,4,100
4,5,150
5,6,150
6,17,50
7,47,50
8,49,50


### 2. 3일 체험 신청자 출입기록 분석

In [ ]:
# 3일 체험 신청자 출입기록
# 입실 : 1
# 퇴실 : 2
trial_access_log.head(5)

,id,checkin,cdate,site_id,user_uuid
0,1719038,1,2023-07-07 07:11:55.201673,49,cacd0adb-2c87-450f-8a00-2b2ea6b8fa89
1,1716702,2,2023-07-06 15:15:58.761284,49,cacd0adb-2c87-450f-8a00-2b2ea6b8fa89
2,1719956,2,2023-07-07 14:10:41.848998,49,cacd0adb-2c87-450f-8a00-2b2ea6b8fa89
3,1716588,1,2023-07-06 13:09:48.758097,49,cacd0adb-2c87-450f-8a00-2b2ea6b8fa89
4,1719649,1,2023-07-07 10:12:42.083352,49,cacd0adb-2c87-450f-8a00-2b2ea6b8fa89


In [ ]:
# 출입 기록 데이터 총 63,708개
print(f"전체 데이터 개수 : {len(trial_access_log)}개\n")

# 출입 id의 고유값 수
print(f"출입 id의 고유값 개수 : {trial_access_log['id'].nunique()}개\n")

# 출입 id의 중복값 수
print(f"출입 id의 중복값 개수 : {trial_access_log['id'].duplicated().sum()}개\n")

# datetime 형태로 변환
trial_access_log['cdate'] = pd.to_datetime(trial_access_log['cdate'])

# 출입 기록이 가장 오래된 날짜 및 시간
print(f"최초의 출입 기록 날짜 : {trial_access_log['cdate'].min()}\n")

# 출입 기록이 가장 최근인 날짜 및 시간
print(f"최근 출입 기록 날짜 : {trial_access_log['cdate'].max()}\n")

# 유저의 고유값 수
print(f"3일체험 신청하고 한번이라도 출입 또는 퇴실한 유저의 수 : {trial_access_log['user_uuid'].nunique()}명\n")

전체 데이터 개수 : 63708개

출입 id의 고유값 개수 : 63349개

출입 id의 중복값 개수 : 359개

최초의 출입 기록 날짜 : 2021-05-02 09:42:13.893455

최근 출입 기록 날짜 : 2024-01-01 14:36:13.987243

3일체험 신청하고 한번이라도 출입 또는 퇴실한 유저의 수 : 6026명



### 3일 체험 신청자 출입기록 분석 결과

- id 컬럼과 user_uuid는 서로 동일하지 않다. user_uuid에 따라서 id가 고정되어 있지 않으며, 출입기록별 id가 새롭게 갱신되는 것으로 확인됨
- id 컬럼의 고유값 개수는 63,349개로 전체 데이터 개수인 63,708보다 적다. 중복된 id가 359개 존재한다는 것을 확인
    - 세부 분석을 진행할 때 id가 중복되는 데이터는 어떤 케이스인지 확인
- 출입 기록 데이터의 날짜는 21년 5월 2일 ~ 24년 1월 1일까지 수집
- 3일체험 신청자는 중복을 제외하고 6,026명이 출입하였음,
    - 출입 로그 데이터이므로 화장실이 밖에 있거나, 식사 및 흡연으로 인해 출입기록이 중복을 포함해서 다수 존재할 수 있음

### 3. 3일체험 신청자 결제 여부 분석

In [ ]:
# 3일 체험 신청자 결제 여부
trial_payment.sample(5)

,is_payment,user_uuid
800,0,2ced92f9-804a-438e-9031-ad7c1fd4e4fe
8248,1,958f0383-00fb-42a6-9d9a-869b41e77d17
1237,1,d736fdf6-84ee-49a2-b60d-4b6d176a328e
9433,0,3d15cfd1-5a6d-4d18-8afe-a9549b6ad6cc
1471,0,8c776576-c5b1-420d-b011-0597af932421


In [ ]:
# 전체 데이터 수
print(f"전체 데이터 수 : {len(trial_payment)}명\n")

# 중복을 제외한 유저의 수
print("중복 제외된 유저의 수 : {trial_payment['user_uuid'].nunique()}명\n")

# 결제한 사람의 수
print(f"결제한 사람의 수 : {sum(trial_payment['is_payment'] == 1)}명. 전체 약 38%\n")

# 결제를 하지 않은 사람의 수
print(f"결제하지 않은 사람의 수 : {sum(trial_payment['is_payment'] == 0)}명. 전체 약 62%")

전체 데이터 수 : 9659명

중복 제외된 유저의 수 : {trial_payment['user_uuid'].nunique()}명

결제한 사람의 수 : 3664명. 전체 약 38%

결제하지 않은 사람의 수 : 5995명. 전체 약 62%


### 3일 체험 신청자 결제 여부 분석 결과
- 전체 데이터는 9,659개, 중복 제외 유저id는 9,624개, 중복되는 유저id가 존재함
    - 여러 번 결제한 내역이 존재할 수 있음
- 결제(1) : 3,664명, 전체 약 38% 유저가 3일 체험 이후 결제함
- 미결제(2) : 5,995명, 전체 약 62% 유저가 3일 체험 이후 결제하지 않음

### 4. 3일 체험 신청 데이터 분석

In [ ]:
# 3일 체험 신청 데이터
trial_register.sample(5)

,trial_date,user_uuid
3692,2022-03-28,6aed9803-4a31-44ed-88e6-e2d09e8f2b76
6623,2022-12-22,f60bf76f-1f22-417c-8500-ff664f432eb5
1272,2021-07-26,b44ab4f1-7f90-4391-a751-3c3e2f0f7c1a
7899,2023-05-09,828b3325-2326-453f-a8c3-c6fbbc8f8055
9614,2023-11-11,1821a7a8-b408-4847-9f33-684093f3c5df


In [ ]:
# trial_date 컬럼을 datetime 타입으로 변환
trial_register['trial_date'] = pd.to_datetime(trial_register['trial_date'])

# 전체 데이터 길이
print(f"전체 데이터 길이 : {len(trial_register)}개\n")

# 중복 제외한 유저의 수
print(f"중복 제외한 유저의 수 : {trial_register['user_uuid'].nunique()}명\n")

# 처음으로 3일 체험 신청한 날짜
print(f"처음으로 3일 체험 신청한 날짜 : {trial_register['trial_date'].min()}\n")

# 마지막으로 3일 체험 신청한 날짜
print(f"마지막으로 3일 체험 신청한 날짜 : {trial_register['trial_date'].max()}")

전체 데이터 길이 : 9659개

중복 제외한 유저의 수 : 9624명

처음으로 3일 체험 신청한 날짜 : 2021-05-01 00:00:00

마지막으로 3일 체험 신청한 날짜 : 2023-12-31 00:00:00


### 3일 체험 신청 데이터 분석 결과
- 데이터 수 불일치 : 전체 데이터는 9,659개, 중복 제외 유저 9,624명으로 유저가 중복되는 오류값이 존재함
    - 오류값 존재하는 이유 : 서버 전산 오류 등 예측불가 오류
- 처음으로 체험 신청한 날 : 21년 5월 1일
- 마지막으로 체험 신청한 날 : 23년 12월 31일

### 5. 3일 체험 신청자 일자별 방문기록 분석

In [ ]:
# 3일 체험 신청자 일자별 방문기록
trial_visit_info.sample(5)

,site_id,date,stay_time,stay_time_second,first_enter_time,last_leave_time,user_uuid
3497,17,2022-09-30,03:28:33.476563,12513,2022-09-30 16:32:35.422524,2022-09-30 20:08:15.952713,f346736f-0f1d-4da9-8c2a-497e20b9295c
1685,2,2022-08-09,00:56:54.049448,3414,2022-08-09 19:41:06.769513,2022-08-09 20:38:00.818961,496d8690-227e-4abc-80fe-22512eb71730
10223,17,2023-08-20,05:52:29.039740,21149,2023-08-20 10:34:44.824152,2023-08-20 16:49:43.222204,9d8ac39d-fcc9-4f11-b696-817122e411c8
7495,4,2023-09-04,00:20:16.390759,1216,2023-09-04 13:29:20.161716,2023-09-04 13:49:36.552475,ebf82182-8026-48e5-8324-0f5446a166ca
10289,17,2023-08-08,01:33:07.905237,5587,2023-08-08 15:43:28.009081,2023-08-08 17:19:39.540221,a7bc4b69-3804-41e3-817b-c5a12f4bf03c


In [ ]:
# 3일체험 신청자 방문기록 전체 중복되는 데이터 확인
print(f"방문기록 완전히 중복되는 데이터 수 : {trial_visit_info.duplicated().sum()}건\n")

# 중복 제거한 데이터
trial_visit_info_drop = trial_visit_info.drop_duplicates()

# datetime 타입 변환
trial_visit_info_drop['date'] = pd.to_datetime(trial_visit_info_drop['date'])
trial_visit_info_drop['stay_time'] = pd.to_datetime(trial_visit_info_drop['stay_time'])

# 오직 시간만 확인하기 위해 stay_time2라는 컬럼 새롭게 생성
trial_visit_info_drop['stay_time2'] = trial_visit_info_drop['stay_time'].dt.time
trial_visit_info_drop['first_enter_time'] = pd.to_datetime(trial_visit_info_drop['first_enter_time'])
trial_visit_info_drop['last_leave_time'] = pd.to_datetime(trial_visit_info_drop['last_leave_time'])

# 3일 체험 신청 후 방문한 유저의 수
print(f"체험 신청하고 한 번이라도 공유오피스에 방문한 유저의 수 : {trial_visit_info_drop['user_uuid'].nunique()}명\n")

# 유저별 최다 방문 횟수
print(f"가장 많이 방문한 유저의 수 : {trial_visit_info_drop['user_uuid'].value_counts()[:1].values}번\n")

# 가장 많이 방문한 유저의 로그 기록
trial_visit_info_drop[trial_visit_info_drop['user_uuid'] == '4eec8463-d053-44a1-942d-f79e90d3a106']

# print("가장 많이 방문한 유저의 로그 기록")
# display(trial_visit_info_drop.sort_values('first_enter_time', ascending=True))

방문기록 완전히 중복되는 데이터 수 : 48건

체험 신청하고 한 번이라도 공유오피스에 방문한 유저의 수 : 6534명

가장 많이 방문한 유저의 수 : [6]번



,site_id,date,stay_time,stay_time_second,first_enter_time,last_leave_time,user_uuid,stay_time2
1785,5,2022-08-29,2025-03-05 03:30:59.135425,12659,2022-08-29 00:00:00.000000,2022-08-29 03:32:47.587406,4eec8463-d053-44a1-942d-f79e90d3a106,03:30:59.135425
1786,5,2022-08-29,2025-03-05 01:39:03.506200,5943,2022-08-29 22:06:56.404738,2022-08-29 23:59:59.000000,4eec8463-d053-44a1-942d-f79e90d3a106,01:39:03.506200
1787,5,2022-08-30,2025-03-05 00:20:15.448360,1215,2022-08-30 00:00:00.000000,2022-08-30 00:20:15.448360,4eec8463-d053-44a1-942d-f79e90d3a106,00:20:15.448360
1788,5,2022-08-30,2025-03-05 08:56:53.479324,32213,2022-08-30 00:23:04.126849,2022-08-30 17:47:56.899029,4eec8463-d053-44a1-942d-f79e90d3a106,08:56:53.479324
1789,5,2022-09-01,2025-03-05 08:01:33.889395,28893,2022-09-01 09:30:19.035640,2022-09-01 18:00:35.400705,4eec8463-d053-44a1-942d-f79e90d3a106,08:01:33.889395
1790,5,2022-08-31,2025-03-05 07:55:12.033467,28512,2022-08-31 09:25:06.466310,2022-08-31 18:02:49.964377,4eec8463-d053-44a1-942d-f79e90d3a106,07:55:12.033467


# 2. 데이터 결합
시각화 및 데이터 분포를 위한 탐색적 데이터 분석(EDA)을 진행하기 전에 데이터간의 결합이 가능한지 확인 후 데이터 결합을 진행
## 2-1. 결제데이터, 3일 체험 등록데이터 결합
결제데이터(trial_payment)와 등록데이터(trial_register)를 결합하여 하나의 데이터프레임으로 구성한다.
### 1) 결제 데이터 중복 제거 처리

In [ ]:
# 결제 데이터는 전체 행이 중복되는 경우만 존재하므로 단순하게 drop_duplicates() 활용해서 중복을 제거
trial_payment_drop = trial_payment.drop_duplicates()
trial_payment_drop # 총 9,624명의 유저 확인

,is_payment,user_uuid
0,0,2b251333-8676-4c11-a736-dcf2350f8821
1,0,e111619a-0975-451b-9a4a-bc8aea7b7b84
2,1,4a184795-b056-4572-a874-644f68609ea3
3,0,2ba8ab19-2d40-4423-ad04-f0f9ca814871
4,0,1d49ba36-6c23-405b-9514-aa7f4aeceff0
...,...,...
9654,0,d7599df4-1e5a-4f5e-97ce-c42047bfd87c
9655,0,43263092-3b28-4817-9fa9-4205ad3097fe
9656,0,51a40f33-1027-4544-9b95-45bca7c104fb
9657,0,3e649531-bf5b-4b99-84e5-ca3e0e647d0c


### 2) 3일 체험 등록데이터 중복 제거 처리

In [ ]:
# 1. 날짜와 유저id가 완전히 동일한 데이터 중복 제거를 진행
# 이유 : 시간까지 확인되진 않으므로 날짜가 동일한 애들은 전부 삭제해줘도 무관
trial_register_drop = trial_register.drop_duplicates()

# 9,631명의 데이터 존재하지만, 아직 중복되는 값이 더 존재함
print(f"전체 행 기준 중복 제거 후 데이터 개수 : {len(trial_register_drop)}명\n")

# 날짜가 서로 다르지만 유저id가 중복되는 데이터 개수
print(f"날짜가 다르지만, 유저id가 중복되는 데이터 개수 : {len(trial_register_drop[trial_register_drop['user_uuid'].duplicated()])}명")

전체 행 기준 중복 제거 후 데이터 개수 : 9631명

날짜가 다르지만, 유저id가 중복되는 데이터 개수 : 7명


- 전체 행 기준으로 중복을 제거
- 유저id는 동일하지만, 날짜가 다른 중복데이터가 존재
- 유저id를 기준으로 늦게 등록한 데이터를 남겨두고, 이전 중복데이터를 삭제
    - 과거에 무료 이용권을 등록 취소하고, 다시 며칠 뒤에 재등록함
    - 즉, 가장 최신 데이터만 남기도록 중복데이터 제거

In [ ]:
# 유저 아이디를 기준으로 중복데이터 확인 후, 가장 최신의 데이터만 남기고 중복데이터를 삭제
trial_register_drop2 = trial_register_drop.drop_duplicates(subset=['user_uuid'], keep='last')

# 최종 데이터 결과
print(f"최종 중복 제거된 trial_register 인원의 수 : {len(trial_register_drop2)}명")

최종 중복 제거된 trial_register 인원의 수 : 9624명


### 3) 데이터 병합

In [ ]:
# 중복 제거된 trial_payment 데이터와 trial_register 데이터를 user_uuid 기준으로 병합
register_payment_merged = trial_payment_drop.merge(trial_register_drop2[['user_uuid', 'trial_date']], on='user_uuid', how='left')

# 병합 후 전체 행 중복 여부 확인
print(f"병합 후 데이터 총 개수 확인 : {len(register_payment_merged)}개\n")

# 병합 후 데이터 총 개수 확인
print(f"병합 후 전체 행 중복 여부 확인 : {register_payment_merged.duplicated().sum()}개\n")

# 데이터 프레임 확인
print("병합 된 데이터 예시5개 확인")
display(register_payment_merged.sample(5))

병합 후 데이터 총 개수 확인 : 9624개

병합 후 전체 행 중복 여부 확인 : 0개

병합 된 데이터 예시5개 확인


,is_payment,user_uuid,trial_date
8115,1,7faa562f-b924-436f-b602-a950f56a8f5e,2023-06-11
2249,0,584dca6e-8a34-4616-aeb5-fd1db843c270,2021-11-04
5014,0,81568411-ef95-4187-88e4-aea291100559,2022-09-13
2404,1,06492eb4-8140-4a76-abb2-153e5ed54120,2021-11-22
955,1,10c74c15-e0b7-4c30-9001-dc185a030818,2021-06-30


## 2-2. 출입기록 병합
2-1에서 병합된 등록, 결제여부 데이터(`register_payment_merged`)와 출입기록 데이터(`trial_access_log`)를 병합하여 하나의 데이터 프레임으로 생성
### 1) 출입기록 날짜 데이터 분할
`cdate` 컬럼을 `year(연도)`, `month(월)`, `day(일자)`, `

In [ ]:
# 원본 데이터 카피 후 병합 진행
trial_access_log2 = trial_access_log.copy()

# 날짜, 시간 전부 각개의 컬럼으로 변경, EDA 시각화 활용 예정
trial_access_log2['year'] = trial_access_log2['cdate'].dt.year
trial_access_log2['month'] = trial_access_log2['cdate'].dt.month
trial_access_log2['day'] = trial_access_log2['cdate'].dt.day
trial_access_log2['time'] = trial_access_log2['cdate'].dt.time

# 불필요한 id컬럼 삭제, cdate는 데이터 내림,오름차순에 사용
trial_access_log2 = trial_access_log2.drop(columns=['id'])

# 유저의 출입 기록 확인
trial_access_log2[trial_access_log2['user_uuid'].duplicated()][:10].sort_values('cdate', ascending=True)

,checkin,cdate,site_id,user_uuid,year,month,day,time
3,1,2023-07-06 13:09:48.758097,49,cacd0adb-2c87-450f-8a00-2b2ea6b8fa89,2023,7,6,13:09:48.758097
1,2,2023-07-06 15:15:58.761284,49,cacd0adb-2c87-450f-8a00-2b2ea6b8fa89,2023,7,6,15:15:58.761284
5,1,2023-07-06 15:28:39.279318,49,cacd0adb-2c87-450f-8a00-2b2ea6b8fa89,2023,7,6,15:28:39.279318
6,2,2023-07-06 18:17:33.011298,49,cacd0adb-2c87-450f-8a00-2b2ea6b8fa89,2023,7,6,18:17:33.011298
8,2,2023-07-07 09:22:36.745763,49,cacd0adb-2c87-450f-8a00-2b2ea6b8fa89,2023,7,7,09:22:36.745763
4,1,2023-07-07 10:12:42.083352,49,cacd0adb-2c87-450f-8a00-2b2ea6b8fa89,2023,7,7,10:12:42.083352
9,2,2023-07-07 11:09:52.861538,49,cacd0adb-2c87-450f-8a00-2b2ea6b8fa89,2023,7,7,11:09:52.861538
7,1,2023-07-07 14:08:25.192590,49,cacd0adb-2c87-450f-8a00-2b2ea6b8fa89,2023,7,7,14:08:25.192590
2,2,2023-07-07 14:10:41.848998,49,cacd0adb-2c87-450f-8a00-2b2ea6b8fa89,2023,7,7,14:10:41.848998
10,1,2023-07-07 14:24:21.501446,49,cacd0adb-2c87-450f-8a00-2b2ea6b8fa89,2023,7,7,14:24:21.501446


#### 한 명의 유저가 일자별로 출입한 기록을 일부 확인한 결과
- 출입 또는 퇴실한 기록이 없을 수 있음
    - 누군가 앞에서 문을 열어주고 뒤따라 들어간 경우

### 2) 유저별 방문 횟수 구하기
유저별로 무료 이용권을 통해 공유오피스에 방문한 횟수 구하기

In [ ]:
# 유저ID 별로 그룹화 진행 및 날짜의 고유값 개수 확인
trial_access_log_daycnt = trial_access_log2.groupby('user_uuid')['day'].nunique().reset_index()

# 출입한 날이 많은 순서대로 확인
trial_access_log_daycnt = trial_access_log_daycnt.sort_values('day', ascending=False)

# 가장 많이 방문한 상위 3명 확인
trial_access_log_daycnt.head(3)

,user_uuid,day
1298,351979ce-62c9-4170-ae4f-3fe60a2492ed,4
5409,e53dbfe5-3f4f-4243-8423-5a1ef3ebfcc3,4
1906,4eec8463-d053-44a1-942d-f79e90d3a106,4


### 3) 유저별 공유오피스 입실, 퇴실 총 횟수 구하기

In [ ]:
# 각 유저들의 입장, 퇴실 전체 합친 수
access_all_count = trial_access_log['user_uuid'].value_counts()

# 유저id 마다 입장, 퇴실 횟수를 합친 데이터를 access_cnt 컬럼에 입력
trial_access_log_daycnt['access_cnt'] = trial_access_log_daycnt['user_uuid'].map(access_all_count)

# 입장, 퇴실 가장 많이한 상위 3명 확인
trial_access_log_daycnt.sort_values('access_cnt', ascending=False)[:10]

,user_uuid,day,access_cnt
3524,948ae0cc-bc5a-4bd3-89ce-92b2a5c34710,3,126
1844,4c5b26a3-24d4-4b8f-a591-be80271c0aeb,3,108
5263,dfcf88d6-b293-4598-89c0-6598865e3fd9,3,80
2179,5a8723ea-4207-4d94-ab06-d7f7d7de8b61,3,76
1809,4b148aaf-0149-4d63-81ba-0cae91598ff2,3,76
4790,ca76e5bd-5e76-47ff-9ef9-f0fac8b64edd,2,75
1684,4622f625-f93d-4f65-9e9a-e7b24171da62,3,72
2248,5d4058cb-528a-45b1-a2dd-7cf2cc6c401a,3,67
3278,891127aa-959d-4788-a836-dfcc9cf5bfdf,3,67
1065,29787639-0033-4372-acb8-5400e40f9ddd,3,66


### 4) register_payment_merged 데이터와 병합
`register_payment_merged` 데이터에 존재하는 유저가 한 번이라도 공유오피스에 방문하지 않았으면 `day` 와 `access_cnt` 컬럼을 **null**으로 채워준다.

In [ ]:
# register_payment_merged 데이터
access_merged = register_payment_merged.merge(trial_access_log_daycnt[['day', 'access_cnt', 'user_uuid']], on='user_uuid', how='left')

In [ ]:
access_merged[access_merged['day'].isna()]

,is_payment,user_uuid,trial_date,day,access_cnt
2,1,4a184795-b056-4572-a874-644f68609ea3,2023-12-21,NaN,NaN
5,0,283a7a94-8d9e-477a-beb5-29d4d25e4c10,2023-12-22,NaN,NaN
10,0,3fabc6c1-b2d4-4c30-ba2c-f013c34278bd,2023-12-23,NaN,NaN
21,1,552cf9a3-0038-494c-bc9f-1526b5ba3a29,2023-12-24,NaN,NaN
24,0,6f074d46-3c75-4551-b9e6-e1014bbe5d14,2023-12-24,NaN,NaN
...,...,...,...,...,...
9602,1,7f733742-2b31-4f09-b256-696b611803c7,2023-11-15,NaN,NaN
9605,1,544a3e9e-24e4-4a64-bd5d-da0acaec223a,2023-11-15,NaN,NaN
9613,0,42f82dc9-53c0-44d5-8f55-36bd1a198213,2023-11-14,NaN,NaN
9618,0,6301b10b-9ef2-471d-a179-a6d327bebd9c,2023-11-17,NaN,NaN


## 2-3. 체류시간 합치기
`trial_visit_info`데이터에서 확인이 가능한 각 유저별의 **체류시간(stay_time_second)** 의 총 합을 구해서 `access_merged` 데이터에 **총 체류시간(total_stay_time)** 컬럼을 새롭게 제작한다.
### 1) 총 체류시간(total_stay_time) 계산 방법
- trial_visit_info 데이터에서 각 유저별로 stay_time_second 값을 전부 합산
- 합산된 값을 second(초)단위로 보면 좋을지, 아니면 시:분:초 단위로 보는게 좋을지 팀원들이랑 상의해보기
    - 초 단위로 보면 모델 전처리 과정이 심플해지는 장점 존재
    - 대신 초 단위로 보게 된다면 가장 체류시간이 적은 사람은 9초이고, 체류시간이 긴 사람은 69시간이므로 범위가 너무 넓어지게 된다는 단점이 존재함

In [ ]:
# 총 체류시간 내림차순으로 확인
total_stay_time = trial_visit_info_drop.groupby('user_uuid')['stay_time_second'].sum().reset_index().sort_values('stay_time_second', ascending=False)

total_stay_time

,user_uuid,stay_time_second
5871,e5e8feb2-5c4f-4b48-899d-f46d7a484d58,249203
5162,c9fb93af-63dc-48a2-ac11-bfa995bda87d,195305
2564,62e10dea-aeed-4c2e-a3f1-94bb0bcbbd15,176127
423,0ed8949e-f781-4f90-bec9-80c00f77f361,171005
1937,4acf941b-b706-4891-a592-7e04784a1cc4,162920
...,...,...
5968,e97559d0-555f-4389-b863-f7299a7b2fee,51
4952,c07624ad-447e-4e51-b10f-5d987e762d23,21
5706,e016705b-3a61-4d70-959e-e593fd3d171e,16
2653,661d6e27-bb63-42dc-97b1-3662cb0bd30f,11


- 3일 무료 체험권으로 가장 오랫동안 공유오피스에 머무른 유저는 249203초, 즉 69시간정도 공유오피스에 있었다.
- 반면 가장 체류시간이 적은 유저는 약 9초 정도 기록이 되어있다.

### 2) 총 체류시간 데이터 합치기
앞서 데이터를 병합한 `access_merged` 데이터와 `total_stay_time(총 체류시간 계산된 데이터)`를 **user_uuid** 기준으로 병합

In [ ]:
# access_merged 데이터와 total_stay_time 데이터 합치기
staytime_merged = access_merged.merge(total_stay_time[['user_uuid', 'stay_time_second']],
                                      on='user_uuid', how='left')

# 결과 확인
staytime_merged.head()

,is_payment,user_uuid,trial_date,day,access_cnt,stay_time_second
0,0,2b251333-8676-4c11-a736-dcf2350f8821,2023-12-21,1.0,12.0,19596.0
1,0,e111619a-0975-451b-9a4a-bc8aea7b7b84,2023-12-21,2.0,4.0,11084.0
2,1,4a184795-b056-4572-a874-644f68609ea3,2023-12-21,NaN,NaN,NaN
3,0,2ba8ab19-2d40-4423-ad04-f0f9ca814871,2023-12-21,1.0,3.0,4775.0
4,0,1d49ba36-6c23-405b-9514-aa7f4aeceff0,2023-12-21,1.0,5.0,5037.0


### 3) 출입 날짜 횟수와 체류시간이 전부 null이 아닌 경우 탐색
전산상 오류로 인하여 **day**, **access_cnt**, **stay_time_second** 컬럼 중에서 일부만 null값이고 다른컬럼은 null이 아닌 데이터가 존재하는지 확인

In [ ]:
# null값 탐색 컬럼 리스트
check_col = ['day', 'access_cnt', 'stay_time_second']

# day, access_cnt, stay_time_second 컬럼에서 일부만 null이 아닌 값을 탐색
filtered_null_df = staytime_merged[staytime_merged[check_col].isna().sum(axis=1).between(1, len(check_col) - 1)]

# 체류시간 기준 내림차순으로 확인
filtered_null_df.sort_values('stay_time_second', ascending=False)

,is_payment,user_uuid,trial_date,day,access_cnt,stay_time_second
7869,0,8cf1fdb1-0ad4-4940-bd26-86444124cd15,2023-05-09,NaN,NaN,82748.0
1145,1,dc44530f-b81a-4fc6-aadc-1ce04187b24d,2021-07-16,NaN,NaN,78472.0
7917,1,a7c1b276-1cca-4175-bac7-1922ff1bd25f,2023-05-15,NaN,NaN,78169.0
1872,0,83055aed-1d76-4805-bcaa-c4bab943fe0a,2021-09-23,NaN,NaN,66878.0
1268,1,b4f14ff4-b825-4797-8788-38bea3b6a802,2021-07-27,NaN,NaN,63822.0
...,...,...,...,...,...,...
4684,0,d1e26fac-d4fc-46e2-907c-35fbc7d6517b,2022-08-08,NaN,NaN,121.0
5921,1,c76a8e6c-d93c-4d02-9fea-3837ec6c5dd6,2022-10-13,NaN,NaN,51.0
4846,0,fe55f5e4-4027-4860-b3c7-1338b18846d0,2022-08-25,1.0,9.0,NaN
7263,0,070ff794-9409-447c-9889-f463e1acb1a3,2023-03-02,1.0,1.0,NaN


#### null값 탐색결과
514개의 데이터에 문제가 있는 것으로 확인
- day, access_cnt 두 컬럼이 null이면 stay_time_second는 null값이 아님
- day, access_cnt가 null이 아니면 stay_time_second가 null으로 표현됨
#### 문제 원인
유저의 로그데이터가 제대로 기록되지 않아서 `trial_visit_info`에는 출입한 기록이 있지만,
<br>`trial_access_log`에는 출입한 기록이 없으며 반대의 경우도 존재한다.

In [ ]:
# trial_visit_info에는 존재하는 유저
print("trial_visit_info에는 존재하는 유저")
display(trial_visit_info[trial_visit_info['user_uuid'] == 'a7c1b276-1cca-4175-bac7-1922ff1bd25f'])

# trial_access_log에는 존재하지 않는 유저
print("trial_access_log에는 존재하지 않는 유저")
display(trial_access_log[trial_access_log['user_uuid'] == 'a7c1b276-1cca-4175-bac7-1922ff1bd25f'])


trial_visit_info에는 존재하는 유저


,site_id,date,stay_time,stay_time_second,first_enter_time,last_leave_time,user_uuid
11229,2,2023-05-18,21:42:49.412475,78169,2023-05-18 00:00:00,2023-05-18 21:51:29.273331,a7c1b276-1cca-4175-bac7-1922ff1bd25f


trial_access_log에는 존재하지 않는 유저


,id,checkin,cdate,site_id,user_uuid


### 4) 해결방안 - KNN 대치법(최근접 대치법)
#### 해결 방안
KNN 알고리즘(K-최근접 이웃) 대치법을 사용해서 null값을 채워준다.
- day, access_cnt가 **null**, stay_time_second는 **null이 아닐 때**
    - stay_time_second가 비슷한 데이터의 인접값을 파악해서 day, access_cnt 값을 채워준다.
- stay_time_second가 **null**, day, access_cnt가 **null이 아닐 때**
    - day, access_cnt가 비슷한 데이터의 인접값을 파악해서 stay_time_second 값을 채워준다.
    
#### k 값 결정
k를 5~10 사이의 값으로 설정할 경우 체류시간이 null인 데이터 값이 비정상적으로 높아지는 현상을 발견
- k값을 100으로 설정하여 해당 문제를 해결
- k값은 낮을수록 계산비용이 감소하지만, 해당 데이터셋은 계산속도가 빠르게 작동하므로 k값을 높게 설정

In [ ]:
# KNN 알고리즘 불러오기
from sklearn.impute import KNNImputer
import numpy as np
import pandas as pd

# 비교할 데이터
target_col = ['day', 'access_cnt', 'stay_time_second']

# 데이터 복사해서 KNN 알고리즘 적용
staytime_merged_copy = staytime_merged.copy()

# 모든 target_col이 null인 행을 제외하고 KNN 적용
df_filtered = staytime_merged_copy.dropna(subset=target_col, how='all').copy()

# 인덱스 초기화 (KNN 후 업데이트 오류 방지)
df_filtered = df_filtered.reset_index()

# KNN 적용 (k 값 조정)
imputer = KNNImputer(n_neighbors=100)  # 10~20이 적절
df_imputed = pd.DataFrame(imputer.fit_transform(df_filtered[target_col]), columns=target_col)

# 원래 인덱스를 유지하면서 업데이트
staytime_merged_copy.loc[df_filtered['index'], target_col] = df_imputed.values

# day, access_cnt, stay_time_second 는 각각 날짜와 출입/퇴실 횟수, 체류시간(초) 를 나타내므로 반올림하여 정수형으로 나타낸다.
# 반올림 후 NaN 허용하는 'Int64' 타입으로 변환 (NaN 유지 가능)
staytime_merged_copy['day'] = round(staytime_merged_copy['day']).astype('Int64')
staytime_merged_copy['access_cnt'] = round(staytime_merged_copy['access_cnt']).astype('Int64')
staytime_merged_copy['stay_time_second'] = round(staytime_merged_copy['stay_time_second']).astype('Int64')

# 결과 확인
display(staytime_merged_copy.head())

# 결측치 수 확인
print("컬럼별 결측치 수 확인")
print(staytime_merged_copy.isna().sum())

,is_payment,user_uuid,trial_date,day,access_cnt,stay_time_second
0,0,2b251333-8676-4c11-a736-dcf2350f8821,2023-12-21,1,12,19596
1,0,e111619a-0975-451b-9a4a-bc8aea7b7b84,2023-12-21,2,4,11084
2,1,4a184795-b056-4572-a874-644f68609ea3,2023-12-21,<NA>,<NA>,<NA>
3,0,2ba8ab19-2d40-4423-ad04-f0f9ca814871,2023-12-21,1,3,4775
4,0,1d49ba36-6c23-405b-9514-aa7f4aeceff0,2023-12-21,1,5,5037


컬럼별 결측치 수 확인
is_payment             0
user_uuid              0
trial_date             0
day                 3087
access_cnt          3087
stay_time_second    3087
dtype: int64


In [ ]:
# null값이 전부 해결된 병합 데이터
display(staytime_merged_copy)
print("컬럼별 결측치 개수")
print(staytime_merged_copy.isna().sum())

,is_payment,user_uuid,trial_date,day,access_cnt,stay_time_second
0,0,2b251333-8676-4c11-a736-dcf2350f8821,2023-12-21,1,12,19596
1,0,e111619a-0975-451b-9a4a-bc8aea7b7b84,2023-12-21,2,4,11084
2,1,4a184795-b056-4572-a874-644f68609ea3,2023-12-21,<NA>,<NA>,<NA>
3,0,2ba8ab19-2d40-4423-ad04-f0f9ca814871,2023-12-21,1,3,4775
4,0,1d49ba36-6c23-405b-9514-aa7f4aeceff0,2023-12-21,1,5,5037
...,...,...,...,...,...,...
9619,0,d7599df4-1e5a-4f5e-97ce-c42047bfd87c,2023-11-17,2,32,70443
9620,0,43263092-3b28-4817-9fa9-4205ad3097fe,2023-11-17,<NA>,<NA>,<NA>
9621,0,51a40f33-1027-4544-9b95-45bca7c104fb,2023-11-17,2,13,87966
9622,0,3e649531-bf5b-4b99-84e5-ca3e0e647d0c,2023-11-18,1,6,27128


컬럼별 결측치 개수
is_payment             0
user_uuid              0
trial_date             0
day                 3087
access_cnt          3087
stay_time_second    3087
dtype: int64


## 2-4. 공유오피스 지점 정보 데이터 병합
### 1) trial_visit_info 유저들의 site_id 정보 합치기
전산상의 오류로 인하여 발생하는 문제인 일부 유저의 결측치 문제로 인하여
<br>trial_visit_info 데이터와 trial_access_log 데이터에 방문한 기록이 한 개라도 존재한다면, 유저들의 site_id를 병합해서 입력해준다.

In [ ]:
# staytime_merged_copy 데이터와 trial_visit_info의 유저별 site_id 정보 병합하기
trial_visit_siteid = trial_visit_info.drop_duplicates(subset=['user_uuid'])[['site_id', 'user_uuid']]

staytime_siteid_merge = staytime_merged_copy.merge(trial_visit_siteid[['site_id', 'user_uuid']], on='user_uuid', how='left')

staytime_siteid_merge.head()

,is_payment,user_uuid,trial_date,day,access_cnt,stay_time_second,site_id
0,0,2b251333-8676-4c11-a736-dcf2350f8821,2023-12-21,1,12,19596,49.0
1,0,e111619a-0975-451b-9a4a-bc8aea7b7b84,2023-12-21,2,4,11084,2.0
2,1,4a184795-b056-4572-a874-644f68609ea3,2023-12-21,<NA>,<NA>,<NA>,NaN
3,0,2ba8ab19-2d40-4423-ad04-f0f9ca814871,2023-12-21,1,3,4775,17.0
4,0,1d49ba36-6c23-405b-9514-aa7f4aeceff0,2023-12-21,1,5,5037,17.0


In [ ]:
staytime_siteid_merge.isna().sum()

is_payment             0
user_uuid              0
trial_date             0
day                 3087
access_cnt          3087
stay_time_second    3087
site_id             3090
dtype: int64

### 2) trial_access_log 유저들의 site_id 정보 합치기
trial_visit_info 데이터에 존재하지 않는 유저들의 site_id 입력
- site_id가 입력되지 않은 3개 데이터 탐색
- 해당 데이터의 실제 site_id 값을 찾아서 조건식으로 입력

In [ ]:
# 결합 후 site_id가 null 이면서, day는 채워져있는 데이터 확인
null3_df = staytime_siteid_merge[(staytime_siteid_merge['site_id'].isna()) & (staytime_siteid_merge['day'].notna())]

null3_df

,is_payment,user_uuid,trial_date,day,access_cnt,stay_time_second,site_id
4846,0,fe55f5e4-4027-4860-b3c7-1338b18846d0,2022-08-25,1,9,21982,NaN
7263,0,070ff794-9409-447c-9889-f463e1acb1a3,2023-03-02,1,1,18591,NaN
8915,0,06ed9e12-7ed9-462b-9c1b-3c3fc4e229f9,2023-09-06,2,19,53595,NaN


#### site_id 3개 결측치 데이터 유저id
- fe55f5e4-4027-4860-b3c7-1338b18846d0
- 070ff794-9409-447c-9889-f463e1acb1a3
- 06ed9e12-7ed9-462b-9c1b-3c3fc4e229f9
#### 위 데이터들을 trial_access_log 데이터에서 site_id를 찾아서 staytime_siteid_merge 데이터에 입력

In [ ]:
print(trial_access_log[trial_access_log['user_uuid'] == 'fe55f5e4-4027-4860-b3c7-1338b18846d0']['site_id'][:1])
print(trial_access_log[trial_access_log['user_uuid'] == '070ff794-9409-447c-9889-f463e1acb1a3']['site_id'][:1])
print(trial_access_log[trial_access_log['user_uuid'] == '06ed9e12-7ed9-462b-9c1b-3c3fc4e229f9']['site_id'][:1])

42861    5
Name: site_id, dtype: int64
59887    17
Name: site_id, dtype: int64
32792    4
Name: site_id, dtype: int64


In [ ]:
staytime_siteid_merge.loc[
    staytime_siteid_merge['user_uuid'] == 'fe55f5e4-4027-4860-b3c7-1338b18846d0', 'site_id'] = 5

staytime_siteid_merge.loc[
    staytime_siteid_merge['user_uuid'] == '070ff794-9409-447c-9889-f463e1acb1a3', 'site_id'] = 17

staytime_siteid_merge.loc[
    staytime_siteid_merge['user_uuid'] == '06ed9e12-7ed9-462b-9c1b-3c3fc4e229f9', 'site_id'] = 4

staytime_siteid_merge.isna().sum()

is_payment             0
user_uuid              0
trial_date             0
day                 3087
access_cnt          3087
stay_time_second    3087
site_id             3087
dtype: int64

### 3) 공유오피스 지점 정보가 결합된 데이터

In [ ]:
# 지점정보 결합된 데이터
staytime_siteid_merge.head(3)

,is_payment,user_uuid,trial_date,day,access_cnt,stay_time_second,site_id
0,0,2b251333-8676-4c11-a736-dcf2350f8821,2023-12-21,1,12,19596,49.0
1,0,e111619a-0975-451b-9a4a-bc8aea7b7b84,2023-12-21,2,4,11084,2.0
2,1,4a184795-b056-4572-a874-644f68609ea3,2023-12-21,<NA>,<NA>,<NA>,NaN


### 4) 지점별 면적 정보가 결합된 데이터

In [ ]:
# 지점별 정보와 면적 정보를 합치기
staytime_siteid_merge['office_area'] = staytime_siteid_merge['site_id'].map({1:50, 2:100, 3:150, 4:100, 5:150, 6:150, 17:50, 47:50, 49:50})
staytime_siteid_merge.head(3)

,is_payment,user_uuid,trial_date,day,access_cnt,stay_time_second,site_id,office_area
0,0,2b251333-8676-4c11-a736-dcf2350f8821,2023-12-21,1,12,19596,49.0,50.0
1,0,e111619a-0975-451b-9a4a-bc8aea7b7b84,2023-12-21,2,4,11084,2.0,100.0
2,1,4a184795-b056-4572-a874-644f68609ea3,2023-12-21,<NA>,<NA>,<NA>,NaN,NaN


## 2-5. 이용 날짜 정보 입력
한번이라도 방문한 유저가 주말을 포함해서 방문했는지, 평일에만 방문했는지 구분할 수 있는 컬럼을 제작
<br>주말에 방문한적이 있다면, 직장인이 평일에 방문할 시간이 없어서 주말을 이용해 방문했다고 볼 수 있음
<br>반면 평일에만 방문한 유저는 프리랜서 혹은 취준생으로 생각할 수 있음
### 1) 유저별 방문일수 확인하기
주말에 한번이라도 출입하거나 퇴실했다면 weekend컬럼에 1을 넣어주고,
<br>주말에 출입하거나 퇴실한 기록이 없으면 0으로 입력

In [ ]:
# 출입기록 로그에서 cdate 컬럼을 기준으로 요일을 나타내는 컬럼 생성 dayofweek
trial_access_log2['dayofweek'] = trial_access_log2['cdate'].dt.dayofweek

# 요일별로 매핑해주기, 평일은 0, 주말은 1
weekend_map_dict = {0 : 0, 1 : 0, 2 : 0, 3 : 0, 4 : 0, 5 : 1, 6 : 1}

trial_access_log2['weekend'] = trial_access_log2['dayofweek'].map(weekend_map_dict)

# 평일에 따라서 원핫인코딩 진행하여 컬럼 넣어주기
trial_access_log2_ohe = pd.get_dummies(trial_access_log2, columns=['dayofweek'])

# 유저별로 방문한 날짜의 원핫인코딩 된 컬럼에 1을 적용해주기
access_ohe_days = trial_access_log2_ohe.groupby('user_uuid').max().reset_index()

# 유저 id와 요일별 원핫인코딩 컬럼만 사용
access_ohe_days = access_ohe_days[['user_uuid', 'dayofweek_0',
       'dayofweek_1', 'dayofweek_2', 'dayofweek_3', 'dayofweek_4',
       'dayofweek_5', 'dayofweek_6']]

#주말에 출입 또는 퇴실한 기록이 있으면 1이상, 출입한 기록이 없으면 0
accsess_user_weekend = trial_access_log2.groupby('user_uuid')['weekend'].sum().reset_index()

# trial_visit_info 데이터도 동일하게 작업 진행
trial_visit_info_drop['dayofweek'] = trial_visit_info_drop['date'].dt.dayofweek
trial_visit_info_drop['weekend'] = trial_visit_info_drop['dayofweek'].map(weekend_map_dict)

# 평일에 따라서 원핫인코딩 진행하여 컬럼 넣어주기
trial_visit_info_ohe = pd.get_dummies(trial_visit_info_drop, columns=['dayofweek'])

# 유저별로 방문한 날짜의 원핫인코딩 된 컬럼에 1을 적용해주기
visit_ohe_days = trial_visit_info_ohe.groupby('user_uuid').max().reset_index()

# 유저 id와 요일별 원핫인코딩 컬럼만 사용
visit_ohe_days = visit_ohe_days[['user_uuid', 'dayofweek_0',
       'dayofweek_1', 'dayofweek_2', 'dayofweek_3', 'dayofweek_4',
       'dayofweek_5', 'dayofweek_6']]

#주말에 출입 또는 퇴실한 기록이 있으면 1이상, 출입한 기록이 없으면 0
visit_user_weekend = trial_visit_info_drop.groupby('user_uuid')['weekend'].sum().reset_index()

# # 주말이 하루라도 포함되어 있으면 weekend를 1로, 평일만 있으면 0으로 컬럼을 재설정
visit_user_weekend['weekend'] = visit_user_weekend['weekend'].apply(lambda x : 1 if x >= 1 else 0)
accsess_user_weekend['weekend'] = accsess_user_weekend['weekend'].apply(lambda x : 1 if x >= 1 else 0)

print("trial_visit_info 유저 주말 여부")
display(visit_user_weekend.head())
print("trial_visit_info 유저가 무슨 요일에 방문했는지 확인")
display(visit_ohe_days.head())
print("------------------------------")
print("access_log 유저 주말 여부")
display(accsess_user_weekend.head())
print("access_log 유저가 무슨 요일에 방문했는지 확인")
display(access_ohe_days.head())

trial_visit_info 유저 주말 여부


,user_uuid,weekend
0,000590dc-046f-462b-8225-4c81a97b7166,0
1,000be14f-ba0f-4af6-bc2c-7b00dfcbc98c,1
2,0018e22d-a2cc-4e06-bca2-d6a041770f3b,0
3,002606d9-38ca-48f3-910c-cbfada9a9109,1
4,002a7570-90dd-4cab-b0aa-b0e40f7ef27a,0


trial_visit_info 유저가 무슨 요일에 방문했는지 확인


,user_uuid,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
0,000590dc-046f-462b-8225-4c81a97b7166,0,1,0,0,0,0,0
1,000be14f-ba0f-4af6-bc2c-7b00dfcbc98c,0,0,0,1,1,1,0
2,0018e22d-a2cc-4e06-bca2-d6a041770f3b,0,0,1,1,0,0,0
3,002606d9-38ca-48f3-910c-cbfada9a9109,0,0,0,0,0,1,1
4,002a7570-90dd-4cab-b0aa-b0e40f7ef27a,0,0,1,0,1,0,0


------------------------------
access_log 유저 주말 여부


,user_uuid,weekend
0,000590dc-046f-462b-8225-4c81a97b7166,0
1,000be14f-ba0f-4af6-bc2c-7b00dfcbc98c,0
2,0018e22d-a2cc-4e06-bca2-d6a041770f3b,0
3,002606d9-38ca-48f3-910c-cbfada9a9109,1
4,002a7570-90dd-4cab-b0aa-b0e40f7ef27a,0


access_log 유저가 무슨 요일에 방문했는지 확인


,user_uuid,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
0,000590dc-046f-462b-8225-4c81a97b7166,0,1,0,0,0,0,0
1,000be14f-ba0f-4af6-bc2c-7b00dfcbc98c,0,0,0,1,1,0,0
2,0018e22d-a2cc-4e06-bca2-d6a041770f3b,0,0,1,1,0,0,0
3,002606d9-38ca-48f3-910c-cbfada9a9109,0,0,0,0,0,1,1
4,002a7570-90dd-4cab-b0aa-b0e40f7ef27a,0,0,1,0,0,0,0


### 2) staytime_siteid_merge 데이터에 주말여부 입력하기

In [ ]:
# staytime_siteid_merge에 하나씩 병합하기
weekday_merged = staytime_siteid_merge.merge(visit_user_weekend[['weekend', 'user_uuid']], on='user_uuid', how='left')
weekday_merged.head(3)

,is_payment,user_uuid,trial_date,day,access_cnt,stay_time_second,site_id,office_area,weekend
0,0,2b251333-8676-4c11-a736-dcf2350f8821,2023-12-21,1,12,19596,49.0,50.0,0.0
1,0,e111619a-0975-451b-9a4a-bc8aea7b7b84,2023-12-21,2,4,11084,2.0,100.0,0.0
2,1,4a184795-b056-4572-a874-644f68609ea3,2023-12-21,<NA>,<NA>,<NA>,NaN,NaN,NaN


In [ ]:
# access 데이터를 결합하고 null값이 존재하는 데이터 확인
weekday_merged2 = weekday_merged.merge(accsess_user_weekend[['user_uuid', 'weekend']], on='user_uuid', how='left')
weekday_merged2[weekday_merged2['weekend_x'].isna() & weekday_merged2['weekend_y'].notna()]

,is_payment,user_uuid,trial_date,day,access_cnt,stay_time_second,site_id,office_area,weekend_x,weekend_y
4846,0,fe55f5e4-4027-4860-b3c7-1338b18846d0,2022-08-25,1,9,21982,5.0,150.0,NaN,0.0
7263,0,070ff794-9409-447c-9889-f463e1acb1a3,2023-03-02,1,1,18591,17.0,50.0,NaN,0.0
8915,0,06ed9e12-7ed9-462b-9c1b-3c3fc4e229f9,2023-09-06,2,19,53595,4.0,100.0,NaN,0.0


In [ ]:
# null값이 존재하는 데이터에는 access데이터에서 주말이 포함되지 않았음을 확인하고 0을 입력
weekday_merged.loc[
    weekday_merged['user_uuid'] == 'fe55f5e4-4027-4860-b3c7-1338b18846d0', 'weekend'] = 0

weekday_merged.loc[
    weekday_merged['user_uuid'] == '070ff794-9409-447c-9889-f463e1acb1a3', 'weekend'] = 0

weekday_merged.loc[
    weekday_merged['user_uuid'] == '06ed9e12-7ed9-462b-9c1b-3c3fc4e229f9', 'weekend'] = 0

# 결측치의 개수가 동일하게 나오고 있음
weekday_merged.isna().sum()

is_payment             0
user_uuid              0
trial_date             0
day                 3087
access_cnt          3087
stay_time_second    3087
site_id             3087
office_area         3087
weekend             3087
dtype: int64

### 3) 방문요일 데이터 합치기

In [ ]:
dayofweek_merged = weekday_merged.merge(visit_ohe_days[['user_uuid', 'dayofweek_0',
       'dayofweek_1', 'dayofweek_2', 'dayofweek_3', 'dayofweek_4',
       'dayofweek_5', 'dayofweek_6']], on='user_uuid', how='left')

trial_visit 데이터만 병합했으므로 3개의 결측치가 추가로 더 존재함

In [ ]:
# 결측치 개수 확인
dayofweek_merged

,is_payment,user_uuid,trial_date,day,access_cnt,stay_time_second,site_id,office_area,weekend,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
0,0,2b251333-8676-4c11-a736-dcf2350f8821,2023-12-21,1,12,19596,49.0,50.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0,e111619a-0975-451b-9a4a-bc8aea7b7b84,2023-12-21,2,4,11084,2.0,100.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,4a184795-b056-4572-a874-644f68609ea3,2023-12-21,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,2ba8ab19-2d40-4423-ad04-f0f9ca814871,2023-12-21,1,3,4775,17.0,50.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0,1d49ba36-6c23-405b-9514-aa7f4aeceff0,2023-12-21,1,5,5037,17.0,50.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9619,0,d7599df4-1e5a-4f5e-97ce-c42047bfd87c,2023-11-17,2,32,70443,6.0,150.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
9620,0,43263092-3b28-4817-9fa9-4205ad3097fe,2023-11-17,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9621,0,51a40f33-1027-4544-9b95-45bca7c104fb,2023-11-17,2,13,87966,6.0,150.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
9622,0,3e649531-bf5b-4b99-84e5-ca3e0e647d0c,2023-11-18,1,6,27128,3.0,150.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


결측치가 존재하는 행들을 확인 후 각 유저별로 방문한 요일을 적용

In [ ]:
# 결측치 존재하는 유저 확인
display(access_ohe_days[access_ohe_days['user_uuid'] == 'fe55f5e4-4027-4860-b3c7-1338b18846d0'])
display(access_ohe_days[access_ohe_days['user_uuid'] == '070ff794-9409-447c-9889-f463e1acb1a3'])
display(access_ohe_days[access_ohe_days['user_uuid'] == '06ed9e12-7ed9-462b-9c1b-3c3fc4e229f9'])

,user_uuid,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
5980,fe55f5e4-4027-4860-b3c7-1338b18846d0,0,0,0,0,1,0,0


,user_uuid,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
187,070ff794-9409-447c-9889-f463e1acb1a3,0,0,0,1,0,0,0


,user_uuid,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
181,06ed9e12-7ed9-462b-9c1b-3c3fc4e229f9,0,0,1,1,0,0,0


In [ ]:
# null값이 존재하는 데이터에는 access데이터에서 주말이 포함되지 않았음을 확인하고 0을 입력
dayofweek_merged.loc[
    dayofweek_merged['user_uuid'] == 'fe55f5e4-4027-4860-b3c7-1338b18846d0', 'dayofweek_4'] = 1

dayofweek_merged.loc[
    dayofweek_merged['user_uuid'] == '070ff794-9409-447c-9889-f463e1acb1a3', 'dayofweek_3'] = 1

dayofweek_merged.loc[
    dayofweek_merged['user_uuid'] == '06ed9e12-7ed9-462b-9c1b-3c3fc4e229f9', 'dayofweek_2'] = 1

dayofweek_merged.loc[
    dayofweek_merged['user_uuid'] == '06ed9e12-7ed9-462b-9c1b-3c3fc4e229f9', 'dayofweek_3'] = 1

# 결측치의 개수가 동일하게 나오고 있음
dayofweek_merged.isna().sum()

is_payment             0
user_uuid              0
trial_date             0
day                 3087
access_cnt          3087
stay_time_second    3087
site_id             3087
office_area         3087
weekend             3087
dayofweek_0         3090
dayofweek_1         3090
dayofweek_2         3089
dayofweek_3         3088
dayofweek_4         3089
dayofweek_5         3090
dayofweek_6         3090
dtype: int64

day는 null값이 아니고, dayofweek_0이 null값인, 즉 access로 인하여 null로 표현되는 데이터에는 null값을 0으로 변경

In [ ]:
# 특정 조건을 만족하는 행에 대해 fillna(0) 적용
dayofweek_merged.loc[
    dayofweek_merged['day'].notna() & dayofweek_merged['dayofweek_0'].isna(),
    :
] = dayofweek_merged.loc[
    dayofweek_merged['day'].notna() & dayofweek_merged['dayofweek_0'].isna(),
    :
].fillna(0)


In [ ]:
dayofweek_merged.isna().sum()

is_payment             0
user_uuid              0
trial_date             0
day                 3087
access_cnt          3087
stay_time_second    3087
site_id             3087
office_area         3087
weekend             3087
dayofweek_0         3087
dayofweek_1         3087
dayofweek_2         3087
dayofweek_3         3087
dayofweek_4         3087
dayofweek_5         3087
dayofweek_6         3087
dtype: int64

## 2-6. 입퇴실 시간 입력하기

In [ ]:
# 시간 데이터 불러오기
visit_info = pd.read_csv('trial_visit_info2.csv')

# 필요한 데이터만 사용
time_df = visit_info[['user_uuid','first_enter_time','last_leave_time']]

# datetime 형변환 및 시간대 추출
time_df['first_enter_time'] = pd.to_datetime(time_df['first_enter_time'])
time_df['last_leave_time'] = pd.to_datetime(time_df['last_leave_time'])
time_df["enter_hour"] = time_df["first_enter_time"].dt.hour
time_df['leave_hour'] = time_df['last_leave_time'].dt.hour

# 중복 유저 제거(가장 처음에 방문한 날짜 데이터 사용)
time_drop = time_df.drop_duplicates(subset=['user_uuid'], keep='last')

# 데이터 병합
time_merged = dayofweek_merged.merge(time_drop, on='user_uuid', how='left')

# 불필요한 데이터 삭제
time_merged = time_merged.drop(columns=['first_enter_time', 'last_leave_time'], axis=1)

time_merged.head(3)

,is_payment,user_uuid,trial_date,day,access_cnt,stay_time_second,site_id,office_area,weekend,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,enter_hour,leave_hour
0,0,2b251333-8676-4c11-a736-dcf2350f8821,2023-12-21,1,12,19596,49.0,50.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,9.0,16.0
1,0,e111619a-0975-451b-9a4a-bc8aea7b7b84,2023-12-21,2,4,11084,2.0,100.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0,15.0
2,1,4a184795-b056-4572-a874-644f68609ea3,2023-12-21,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 5. 피쳐 인코딩 및 스케일링

## 5-1. 피쳐 인코딩
### 1) 원-핫 인코딩
범주형 데이터는 원-핫 인코딩을 진행
- 공유오피스 지점을 나타내는 범주형 데이터이므로 원-핫 인코딩을 진행

In [ ]:
# 데이터 복사
share_office = time_merged.copy()

# 원핫인코딩 진행

# weekend null : 0, 평일 : 1, 주말 2 으로 주말 여부 컬럼을 변경
share_office['weekend'] = share_office['weekend'].apply(lambda x : 2 if x >= 1 else(1 if x == 0 else 0))

# weekend 데이터 원핫인코딩 진행
share_office = pd.get_dummies(share_office, columns=['weekend'])
share_office = pd.get_dummies(share_office, columns=['office_area'])
share_office = pd.get_dummies(share_office, columns=['site_id'])

share_office.sample(5)

,is_payment,user_uuid,trial_date,day,access_cnt,stay_time_second,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,...,office_area_150.0,site_id_1.0,site_id_2.0,site_id_3.0,site_id_4.0,site_id_5.0,site_id_6.0,site_id_17.0,site_id_47.0,site_id_49.0
8197,0,2fe76127-8b83-4c88-9ff9-79f27134fa5e,2023-06-19,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1315,1,787d21b7-b4ac-44ea-822f-d1bf9a5fdb42,2021-07-31,2,22,39738,1.0,1.0,0.0,0.0,...,1,0,0,1,0,0,0,0,0,0
1350,0,af6d6816-47dc-4b8d-8133-4bd820b576bf,2021-08-02,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1288,0,562c4954-465d-4507-b9ad-87bfa61cec8a,2021-07-28,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
6231,0,679b9c05-099d-40cf-8d69-bc2a8d0864f4,2022-11-26,1,8,27226,0.0,0.0,0.0,0.0,...,1,0,0,0,0,1,0,0,0,0


### 2) 사이클릭 인코딩
시계열 주기를 갖는 데이터는 사이클릭 인코딩을 진행

In [ ]:
# 시간(hour) 열을 사인 & 코사인 변환
share_office['enter_hour_sin'] = np.sin(2 * np.pi * share_office['enter_hour'] / 24)
share_office['enter_hour_cos'] = np.cos(2 * np.pi * share_office['enter_hour'] / 24)
share_office['leave_hour_sin'] = np.sin(2 * np.pi * share_office['leave_hour'] / 24)
share_office['leave_hour_cos'] = np.cos(2 * np.pi * share_office['leave_hour'] / 24)

# 방문하지 않은 유저를 구분하기 위해 is_misiing_time 변수 추가
share_office['is_missing_time'] = share_office['enter_hour_sin'].isna().astype(int)

# 시간 데이터 결측치를 대체
share_office['enter_hour_cos'] = share_office['enter_hour_cos'].fillna(1)
share_office['leave_hour_cos'] = share_office['leave_hour_cos'].fillna(1)
share_office['enter_hour_sin'] = share_office['enter_hour_sin'].fillna(0)
share_office['leave_hour_sin'] = share_office['leave_hour_sin'].fillna(0)

share_office.head(3)

,is_payment,user_uuid,trial_date,day,access_cnt,stay_time_second,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,...,site_id_5.0,site_id_6.0,site_id_17.0,site_id_47.0,site_id_49.0,enter_hour_sin,enter_hour_cos,leave_hour_sin,leave_hour_cos,is_missing_time
0,0,2b251333-8676-4c11-a736-dcf2350f8821,2023-12-21,1,12,19596,0.0,0.0,0.0,1.0,...,0,0,0,0,1,0.707107,-0.707107,-0.866025,-0.500000,0
1,0,e111619a-0975-451b-9a4a-bc8aea7b7b84,2023-12-21,2,4,11084,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0.866025,0.500000,-0.707107,-0.707107,0
2,1,4a184795-b056-4572-a874-644f68609ea3,2023-12-21,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0.000000,1.000000,0.000000,1.000000,1


In [ ]:
share_office = share_office.drop(['enter_hour','leave_hour'], axis = 1)

### 2) 데이터 표준화 진행

In [ ]:
scaled_df[['user_uuid', 'enter_hour_sin', 'enter_hour_cos',
           'leave_hour_sin', 'leave_hour_cos', 'is_missing_time']]

,user_uuid,enter_hour_sin,enter_hour_cos,leave_hour_sin,leave_hour_cos,is_missing_time
0,2b251333-8676-4c11-a736-dcf2350f8821,7.071068e-01,-0.707107,-0.866025,-0.500000,0
1,e111619a-0975-451b-9a4a-bc8aea7b7b84,8.660254e-01,0.500000,-0.707107,-0.707107,0
2,4a184795-b056-4572-a874-644f68609ea3,0.000000e+00,1.000000,0.000000,1.000000,1
3,2ba8ab19-2d40-4423-ad04-f0f9ca814871,-7.071068e-01,0.707107,-0.500000,0.866025,0
4,1d49ba36-6c23-405b-9514-aa7f4aeceff0,-7.071068e-01,0.707107,-0.500000,0.866025,0
...,...,...,...,...,...,...
9619,d7599df4-1e5a-4f5e-97ce-c42047bfd87c,7.071068e-01,-0.707107,-0.500000,0.866025,0
9620,43263092-3b28-4817-9fa9-4205ad3097fe,0.000000e+00,1.000000,0.000000,1.000000,1
9621,51a40f33-1027-4544-9b95-45bca7c104fb,1.224647e-16,-1.000000,-0.965926,0.258819,0
9622,3e649531-bf5b-4b99-84e5-ca3e0e647d0c,-2.588190e-01,-0.965926,-0.707107,0.707107,0


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# 표준화 처리 진행할 컬럼 리스트
scale_col = ['access_cnt', 'stay_time_second', 'day']

scaled_df = share_office.copy()
# 특정 컬럼만 표준화 진행
scaled_df[scale_col] = scaler.fit_transform(scaled_df[scale_col])

scaled_df.sample(4)

,is_payment,user_uuid,trial_date,day,access_cnt,stay_time_second,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,...,site_id_5.0,site_id_6.0,site_id_17.0,site_id_47.0,site_id_49.0,enter_hour_sin,enter_hour_cos,leave_hour_sin,leave_hour_cos,is_missing_time
4120,1,feb92e10-0117-4b8b-923b-74bf6cf22a01,2022-06-02,-0.775964,-0.899175,-0.720695,0.0,0.0,0.0,0.0,...,0,0,0,0,0,-0.965926,0.258819,-0.500000,0.866025,0
1104,1,25a81398-d49f-4e39-b719-c4b6aad9fed4,2021-07-13,2.358096,-0.461448,-0.592036,0.0,1.0,1.0,1.0,...,0,0,0,0,0,-0.866025,0.500000,-0.707107,0.707107,0
5102,0,14e700d2-9a2b-4b08-85fd-2bfe16e83a26,2022-09-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0.000000,1.000000,0.000000,1.000000,1
8931,0,d37e265e-5d2f-4658-bed5-377b9f54761c,2023-09-07,-0.775964,-0.680311,0.358923,0.0,0.0,0.0,0.0,...,0,0,0,0,1,-0.866025,-0.500000,-0.965926,0.258819,0


### 3) null 값 0으로 채우기

In [ ]:
# null값으로 되어있던 방문하지 않은 유저들의 연속형 데이터 0으로 채워주기
scaeld_df = scaled_df.fillna(0, axis=0)

# scaled_df['day'] = scaled_df['day'].fillna(0)
# scaled_df['access_cnt'] = scaled_df['access_cnt'].fillna(0)
# scaled_df['stay_time_second'] = scaled_df['stay_time_second'].fillna(0)
scaled_df.head(3)

,is_payment,user_uuid,trial_date,day,access_cnt,stay_time_second,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,...,site_id_5.0,site_id_6.0,site_id_17.0,site_id_47.0,site_id_49.0,enter_hour_sin,enter_hour_cos,leave_hour_sin,leave_hour_cos,is_missing_time
0,0,2b251333-8676-4c11-a736-dcf2350f8821,2023-12-21,-0.775964,0.195142,-0.376638,0.0,0.0,0.0,1.0,...,0,0,0,0,1,0.707107,-0.707107,-0.866025,-0.500000,0
1,0,e111619a-0975-451b-9a4a-bc8aea7b7b84,2023-12-21,0.791066,-0.680311,-0.710421,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0.866025,0.500000,-0.707107,-0.707107,0
2,1,4a184795-b056-4572-a874-644f68609ea3,2023-12-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0.000000,1.000000,0.000000,1.000000,1


In [ ]:
# 결측치가 존재하는 열을 확인
cols_with_na = scaled_df.columns[scaled_df.isna().any()]

# 결측치가 존재하는 행을 전부 0으로 채우기
scaled_df[cols_with_na] = scaled_df[cols_with_na].fillna(0)

In [ ]:
ㅉ# 피쳐 스케일링 완료된 데이터 csv파일로 내보내기
scaled_df.to_csv("share_office_data4.csv", encoding='utf-8-sig')